In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from TModel.TransformerModel import TranscriptionTransformerModel
from Tokenizer.loaderH5 import H5GuitarTokenizer
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
from TUtils import random_string
import lightning.pytorch as pl
from lightning import Trainer
import torch
from TranscriptionDataset import TranscriptionDataset
from TModel.Retnet.TranscriptionModel import TranscriptionRetnetModel
torch.set_float32_matmul_precision('medium')
import importlib
if importlib.util.find_spec('deepspeed'):
    from lightning.pytorch.strategies import DeepSpeedStrategy
    import deepspeed
    from deepspeed.ops.adam import DeepSpeedCPUAdam, FusedAdam

In [2]:
datasetLocation = "Trainsets/S_Tier_1696235954_mTokens400_mNoS5.hdf5"
wandbProject = "TranscriptionModel_Test"
batchSize = 4
num_workers = 2

In [8]:
dataset,pipe = TranscriptionDataset.getDataPipe(
    datasetLocation,
    batchSize,
    batchFirst=True
)
tokenizer = H5GuitarTokenizer(datasetLocation)
train_pipe,test_pipe = pipe.random_split({"train":0.8,"test":0.2},42,total_length=len(dataset))

train_dl = DataLoader(dataset=train_pipe,batch_size=None,num_workers=num_workers)
test_dl = DataLoader(dataset=test_pipe, batch_size=None,num_workers=num_workers)

In [4]:
model = TranscriptionRetnetModel(
    dataset.getVocabSize(),
    tokenizer,
    d_model=512,
    d_ff=2048,
    lr_init=1e-6,
    # embeddingCheckpoint="Models/GuitarToken/Max2Length.ckpt"
)
try:
    torch.compile(model)
except Exception:
    print("Could not compile model with jit")

Could not compile model with jit


In [5]:
# wandb_logger = WandbLogger(project=wandbProject)
# wandb_logger.experiment.config.update(dataset.meta_data)
# wandb_logger.experiment.config["batchSize"] = batchSize

checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    dirpath=f'Models/GuitarTranscription/{random_string(10)}/',
    filename='GuitarTranscriptionModel-{epoch:02d}-{train_loss:.2f}',
    every_n_train_steps=1000,
    save_top_k=3,
    mode='min',
)

# strategy=DeepSpeedStrategy(offload_optimizer=True, allgather_bucket_size=5e8, reduce_bucket_size=5e8)

trainer = Trainer(
    default_root_dir='Models/',
    max_epochs=10,
    # profiler="simple",
    # strategy=strategy,
    # profiler="pytorch",
    # logger=wandb_logger,
    callbacks=[checkpoint_callback],
    # max_time="00:00:05:00",
    precision="bf16-mixed",
)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model=model, train_dataloaders=train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                | Params
------------------------------------------------------
0 | encoder       | RetnetEncoderLayers | 20.5 M
1 | decoder       | RetnetDecoderLayers | 28.4 M
2 | tgt_embedding | Embedding           | 3.1 M 
3 | outputLinear  | Linear              | 3.1 M 
4 | loss          | CrossEntropyLoss    | 0     
------------------------------------------------------
55.1 M    Trainable params
144       Non-trainable params
55.1 M    Total params
220.496   Total estimated model params size (MB)
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

C:\Users\ritwi\Github\MusicTranscription\Tokenizer\Onsets.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  tokens = torch.argmax(torch.nn.functional.softmax(logits), dim=-1).numpy()
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\mir_eval\onset.py:51: UserWarning: Estimated onsets are empty.
  warnings.warn("Estimated onsets are empty.")
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\mir_eval\onset.py:49: UserWarning: Reference onsets are empty.
  warnings.warn("Reference onsets are empty.")
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\lightning\pytorch\trainer\call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [9]:
trainer.test(model,dataloaders=test_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

C:\Users\ritwi\Github\MusicTranscription\Tokenizer\Onsets.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  tokens = torch.argmax(torch.nn.functional.softmax(logits), dim=-1).numpy()
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\mir_eval\onset.py:51: UserWarning: Estimated onsets are empty.
  warnings.warn("Estimated onsets are empty.")
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\mir_eval\onset.py:49: UserWarning: Reference onsets are empty.
  warnings.warn("Reference onsets are empty.")
C:\Users\ritwi\anaconda3\envs\pytorch\Lib\site-packages\lightning\pytorch\trainer\call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer.save_checkpoint("Models/GuitarTranscription/5s400Tokens_1e-5/smallDataset4epochRetnetJamie.ckpt")